In [ ]:
using Distributed
ifelse(nprocs()<2, addprocs(7), nothing);

@everywhere using LinearAlgebra
@everywhere LinearAlgebra.BLAS.set_num_threads(1) # set number of threads for BLAS
# using LinearAlgebra
# LinearAlgebra.BLAS.set_num_threads(8) # set number of threads for BLAS

In [ ]:
using Plots
# using Revise
@everywhere using LatticeQM

# Tutorial: Twisted honeycomb lattices

## Model

### Bands

In [ ]:
n_moire_angle = 11
tz = 0.46

# n_moire_angle = 13
# tz = 0.39

N_el_per_moire = 6.0
nothing

In [ ]:
@time "lattice" lat = Geometries.honeycomb_twisted(n_moire_angle)
# Geometries.smoothdisplaceZ!(lat, 0.045)
plot_supercells = Structure.Lattices.getneighborcells(lat, 1; halfspace=false, innerpoints=true, excludeorigin=false)
p = plot(lat, 3; supercell=plot_supercells, sort="layer", markersize=1.5)
display(p)

@time "valley_op" valley = Operators.valley(lat; spinhalf=false)
@time "graphene_op" H = Operators.graphene(lat; tz=tz, format=:sparse, mode=:nospin) # specifying format is important here!

# filling = 0.5 + 6.0 / hopdim(hops) # for spinful
filling = 0.5 + N_el_per_moire / 2 / hopdim(H) # for spinful
Operators.setfilling!(H, filling; nk=9^2, multimode=:distributed) # less than 20s on Apple M1 single-core

ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=100)
bands = getbands(H, ks, valley; format=:sparse, multimode=:distributed, num_bands=25)

In [ ]:
p1 = plot(bands; size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)
p2 = plot(bands; ylim=(-0.05, 0.05), size=(300, 250), markercolor=:diverging_bkr_55_10_c35_n256, colorbar=true)

plot(p1, p2; size=(600, 200))

In [ ]:
# H = Operators.graphene(lat; tz=tz, format=:sparse, mode=:spinhalf)
# size(H)

In [ ]:
# using SparseArrays
# using SuiteSparseGraphBLAS

# function mcweeny_purification_test(H, max_iter, eps, E_min, E_max)
#     # Scale H so that its eigenvalues are between 0 and 1
#     # E_min, E_max = eigmin(H), eigmax(H)
#     H_scaled = (H - E_min * I) / (E_max - E_min)

#     # Initial guess for the density matrix
#     P = GBMatrix(H_scaled) # copy(H_scaled)
#     # P_old = zero(P)
#     # Pcache1 = zero(P)
#     # Pcache2 = zero(P)

#     for iter in 1:max_iter
#         println("Iteration $iter")
#         P_old = P
#         # mul!(Pcache1, P, P)
#         # mul!(Pcache2, Pcache1, P)
#         # @. P = 3 * Pcache1 - 2 * Pcache2
#         P = 3 * P * P - 2 * P * P * P  # McWeeny purification

#         # Check for convergence
#         if norm(P - P_old, Inf) < eps
#             return P
#         end
#     end

#     @warn "Maximum iterations reached without convergence or commutation"
#     P
# end

# h_k = H(ks[:, 10]) # sparse ~ 1000 x 1000 matrix
# Emin_k = real.(LatticeQM.Eigen.eigmin_sparse(h_k))
# Emax_k = real.(LatticeQM.Eigen.eigmax_sparse(h_k))

# P_k_test = mcweeny_purification_test(h_k, 35, 1e-6, Emin_k, Emax_k)
# comm_test = norm(h_k * P_k_test - P_k_test * h_k, Inf)

In [ ]:
ks = LatticeQM.Structure.regulargrid(; nk=9^2)
Hs = cat((H(k) for k in eachcol(ks))..., dims=3)
size(Hs) |> println
sizeof(Hs)/(1024^2) |> println
Hs = nothing
GC.gc()

In [ ]:
using SparseArrays
using Plots

function mcweeny_rescaled_H(H, alpha, beta, mu=0.0) # 10.1063/1.1559913
    alpha * (mu * I - H) + beta * I # Scale H so that its eigenvalues are between 0 and 1
end

function mcweeny_initial_P(H, Emax, Emin, mu=0.0; beta=0.5)
    @assert Emax > mu > Emin "mu must be between Emin and Emax"
    # beta = 0.5
    alpha = min(beta/(Emax-mu), (1-beta)/(mu-Emin))
    mcweeny_rescaled_H(H, alpha, beta, mu)
end

function mcweeny_step!(P, P_old, Pcache1, Pcache2; eps=1e-6)
    P_old .= P
    mul!(Pcache1, P, P)
    mul!(Pcache2, Pcache1, P)
    @. P = 3 * Pcache1 - 2 * Pcache2
    # P .= 3 * P * P - 2 * P * P * P  # McWeeny purification
    Pcache1 .-= P
    norm(Pcache1, Inf)
end

function mcweeny_purification(H, max_iter, E_min, E_max, mu; eps=1e-6)
    P = Array(mcweeny_initial_P(H, E_max, E_min, mu))
    P_old = zero(P)
    Pcache1 = zero(P)
    Pcache2 = zero(P)
    converged=false
    for iter in 1:max_iter
        # p0 = heatmap(abs.(P), clim=(0,0.5))
        # display(p0)
        # println("Iteration $iter")
        residual = mcweeny_step!(P, P_old, Pcache1, Pcache2; eps=eps)
        
        if residual < eps
            converged=true
            break # check for convergence
        end
    end
    P_old = nothing 
    Pcache1 = nothing 
    Pcache2 = nothing
    GC.gc()

    if !converged
        @warn "Maximum iterations reached without convergence or commutation (max residual = $(max(residuals...)))"
    end
    P
end

function mcweeny_purification_sparse(H, max_iter, E_min, E_max, mu; eps=1e-6)
    P = mcweeny_initial_P(H, E_max, E_min, mu)
    P_2 = P^2
    # Pcache1 = zero(P)
    # Pcache2 = zero(P)
    converged = false
    for iter in 1:max_iter
        # p0 = heatmap(abs.(P), clim=(0,0.5))
        # display(p0)
        println("Iteration $iter")
        # residual = mcweeny_step!(P, P_old, Pcache1, Pcache2; eps=eps)
        P .= 3 .* P_2 - 2 .* P^3
        P_2 .= P^2
        residual = norm(P - P_2, Inf)
        droptol!(P, 3e-3)

        if residual < eps
            converged = true
            break # check for convergence
        end
    end
    # P_old = nothing
    # Pcache1 = nothing
    # Pcache2 = nothing
    GC.gc()

    if !converged
        @warn "Maximum iterations reached without convergence or commutation (max residual = $(residual))"
    end
    P
end


h_k = H(ks[:, 10]) # sparse ~ 1000 x 1000 matrix
Emin_k = real.(LatticeQM.Eigen.eigmin_sparse(h_k))
Emax_k = real.(LatticeQM.Eigen.eigmax_sparse(h_k))
mu = 0.0

using SuiteSparseGraphBLAS

# P_k_test = mcweeny_purification(h_k, 60, Emin_k, Emax_k, 0.0; eps=1e-6)
P_k_test = mcweeny_purification_sparse(h_k, 60, Emin_k, Emax_k, 0.0; eps=1e-6)
# P_k_test = mcweeny_purification_sparse(GBMatrix(h_k), 60, Emin_k, Emax_k, 0.0; eps=1e-6)
comm_test = norm(h_k * P_k_test - P_k_test * h_k, Inf)

In [ ]:
# count elements of P_k_test bigger than 1e-6
println("Sparsity: ", count(!iszero, abs.(P_k_test) .< 3e-3)/length(P_k_test))
println("Sparsity: ", count(!iszero, abs.(P_k_test) .> 0.01) / length(P_k_test))

In [ ]:
using ProgressMeter

function rescaled_H(H, alpha, beta, mu=0.0) # 10.1063/1.1559913
    alpha * (mu * I - H) + beta * I # Scale H so that its eigenvalues are between 0 and 1
end

function initial_P(H, Emax, Emin, mu=0.0; beta=0.5)
    @assert Emax > mu > Emin "mu must be between Emin and Emax"
    # beta = 0.5
    alpha = min(beta / (Emax - mu), (1 - beta) / (mu - Emin))
    Array(rescaled_H(H, alpha, beta, mu))
end

function canonical_compute_powers!(P_new, P, P_2, P_3)
    P .= P_new
    mul!(P_2, P, P)
    mul!(P_3, P_2, P)
    tr(P_2 - P_3), tr(P - P_2)
end

function canonical_step!(P_new, P, P_2, P_3, c; eps=1e-6)
    @assert 0 <= c <= 1 "c must be between 0 and 1"

    if c >= 0.5
        @. P_new = ((1 + c) * P_2 - P_3) / c
    else
        c < 0.5
        @. P_new = ((1 - 2 * c) * P + (1 + c) * P_2 - P_3) / (1 - c)
    end
    P_2 .-= P_new # overwrite P_2!
    norm(P_2, Inf)
end


function gridspectrumbound(Hs)
    emin = min(real.([LatticeQM.Eigen.eigmin_sparse(h_k) for h_k in Hs])...)
    emax = max(real.([LatticeQM.Eigen.eigmax_sparse(h_k) for h_k in Hs])...)
    emin, emax
end

function canonicalpurification_grid(Hs, max_iter, filling=0.5; eps=1e-6, ebounds=nothing)
    if isnothing(ebounds)
        @time E_min, E_max = gridspectrumbound(Hs)
        println("E_min = $E_min, E_max = $E_max")
    else
        E_min, E_max = ebounds
    end

    N = length(Hs) * size(Hs[1], 1)
    mu = real(sum(tr(h_k) for h_k in Hs) / N)

    Ps_new = [initial_P(h_k, E_max, E_min, mu; beta=filling) for h_k in Hs]
    Ps = [zero(P) for P in Ps_new]
    Ps_2 = [zero(P) for P in Ps_new]
    Ps_3 = [zero(P) for P in Ps_new]
    # P = zero(P); P_2 = zero(P); P_3 = zero(P)

    As = zeros(ComplexF64, length(Hs))
    Bs = zeros(ComplexF64, length(Hs))
    residuals = zeros(Float64, length(Hs))

    converged = false
    prog = ProgressThresh(eps; desc="Density matrix search", showspeed=true)

    for iter in 1:max_iter
        # println("Iteration $iter")

        Threads.@threads for i in 1:length(Hs)
            As[i], Bs[i] = canonical_compute_powers!(Ps_new[i], Ps[i], Ps_2[i], Ps_3[i])
        end

        c = abs(sum(As) / sum(Bs))

        Threads.@threads for i in 1:length(Hs)
            residuals[i] = canonical_step!(Ps_new[i], Ps[i], Ps_2[i], Ps_3[i], c; eps=eps)
        end
        residual = max(residuals...)

        ProgressMeter.update!(prog, residual)

        if residual < eps
            converged = true
            break # check for convergence
        end
    end
    ProgressMeter.finish!(prog)
    Ps = nothing
    Ps_2 = nothing
    Ps_3 = nothing
    GC.gc()

    if !converged
        @warn "Maximum iterations reached without convergence or commutation (max residual = $(max(residuals...)))"
    end

    Ps_new
end

In [ ]:
lat2 = Geometries.honeycomb_twisted(6)
H2 = Operators.graphene(lat2; tz=tz, format=:sparse, mode=:spinhalf)

ks = LatticeQM.Structure.regulargrid(; nk=6^2)
Hs = [H2(k) for k in eachcol(ks)]

Ps = canonicalpurification_grid(Hs, 40, 0.5; eps=1e-6)

filling_after = real(sum(tr(P_k) for P_k in Ps)) / (length(Ps) * size(Ps[1], 1))

In [ ]:
ProgressMeter.ijulia_behavior(:clear)

#############################################
# Low-level TightBinding operator product
#############################################

using SparseArrays

function compute_AB_product(A_matrices::Dict{K,T1}, B_matrices::Dict{K,T2}) where {K,T1,T2}
    T3 = promote_type(T1, T2)
    AB_product = Dict{K,T3}()

    for R in keys(A_matrices)
        # AB_product_R = spzeros(eltype(T3), size(A_matrices[R])...)
        AB_product_R = zero(A_matrices[R])

        for R_prime in keys(B_matrices)
            R_diff = R .- R_prime  # Compute R - R'

            if haskey(A_matrices, R_diff)
                AB_product_R += A_matrices[R_diff] * B_matrices[R_prime]
            end
        end

        AB_product[R] = AB_product_R
    end

    return AB_product
end

SparseArrays.droptol!(H::Hops, args...) = SparseArrays.droptol!(H.data, args...)
function SparseArrays.droptol!(A::Dict{K,T}, tol::Real) where {K,T<:SparseMatrixCSC}
    for R in keys(A)
        SparseArrays.droptol!(A[R], tol)
    end
    return A
end


function compute_AB_product(A::Hops, B::Hops) 
    Hops(compute_AB_product(A.data, B.data))
end

#############################################
# Low-level TightBinding operator product
#############################################

import LatticeQM.TightBinding: zerokey, Hops

function initial_P_realspace(H, Emax, Emin, mu=0.0; beta=0.5)
    @assert Emax > mu > Emin "mu must be between Emin and Emax"
    alpha = min(beta / (Emax - mu), (1 - beta) / (mu - Emin))
    D = size(H,1)

    id = Hops(Dict(zerokey(H) => complex(sparse(1.0*I, D, D))))

    alpha * (mu * id - H) + beta * id
end

function canonicalpurification_grid_realspace(H, E_min, E_max, max_iter, filling=0.5; eps=1e-6, drop_tol=1e-4)

    N = size(Hs[1], 1)
    mu = real(tr(H[[0, 0]]) / N)

    P = initial_P_realspace(H, E_max, E_min, mu; beta=filling)
    P_2 = compute_AB_product(P, P)
    P_3 = compute_AB_product(P, P_2)

    zk = zerokey(H)

    converged = false
    prog = ProgressThresh(eps; desc="Density matrix search", showspeed=true)

    residual = Inf

    for iter in 1:max_iter
        # println("Iteration $iter")
    
        # for R in keys(P)
        #     droptol!(P[R], drop_tol)
        #     droptol!(P_2[R], drop_tol)
        #     droptol!(P_3[R], drop_tol)
        # end

        c = abs(tr(P_2[zk] - P_3[zk]) / tr(P[zk] - P_2[zk]))

        @assert 0 <= c <= 1 "c must be between 0 and 1"

        if c >= 0.5
            P = ((1 + c) * P_2 - P_3) * (1/c)
        else
            c < 0.5
            P = ((1 - 2 * c) * P + (1 + c) * P_2 - P_3) * (1/(1 - c))
        end
        droptol!(P, drop_tol)
    
        P_2 = compute_AB_product(P, P)
        droptol!(P_2, drop_tol)
        P_3 = compute_AB_product(P, P_2)
        droptol!(P_2, drop_tol)

        residual = maximum(norm(P_2[R] - P[R], Inf) for R in keys(P))

        ProgressMeter.update!(prog, residual; showvalues = [(:iter, iter)])

        if residual < eps
            converged = true
            break # check for convergence
        end

    end
    ProgressMeter.finish!(prog)
    P_2 = nothing
    P_3 = nothing
    GC.gc()

    if !converged
        @warn "Maximum iterations reached without convergence or commutation (max residual = $residual)"
    end

    P
end

In [ ]:
lat2 = Geometries.honeycomb_twisted(6)
H2 = Operators.graphene(lat2; tz=tz, format=:sparse, mode=:spinhalf)

P = canonicalpurification_grid_realspace(H2, -3.9, 3.1, 20, 0.5; eps=1e-3)

In [ ]:
filling_after = real(tr(P[zerokey(H2)])) / (size(P, 1))
filling_after

In [ ]:
using LinearAlgebra

h_k = H2(ks[:, 10])
@time evals, U = eigen(Hermitian(Array(h_k)))
f_occ = LatticeQM.Utils.fermidirac_0.(evals)

rho = U * Diagonal(f_occ) * U'
U=nothing
real(tr(rho))

In [ ]:
kgrid = Structure.regulargrid(; nk=9^2)
ωs = collect(range(-0.001, length=100, stop=0.001))
ldos = Spectrum.ldos(H, kgrid, ωs; Γ=0.0005, format=:sparse, num_bands=25);

# ldos0 = ldos[1:2:hopdim(H)] + ldos[2:2:hopdim(H)]
plot(lat, ldos; supercell=plot_supercells, clims=(0, maximum(ldos)), markercolor=:inferno, colorbar=true, sort="layer", markersize=1.5)

In [ ]:
@everywhere GC.gc()

# Mean-field iteration

In [ ]:
@time "valley_op" valley = Operators.valley(lat; spinhalf=true)
@time "graphene_op" H = Operators.graphene(lat; tz=tz, format=:sparse, mode=:spinhalf) # specifying format is important here!

# filling = 0.5 + 6.0 / hopdim(hops) # for spinful 
filling = 0.5 + N_el_per_moire / hopdim(H) # for spinful 
μ = Operators.setfilling!(H, filling; nk=6^2) # less than 20s on Apple M1 single-core 

ks = kpath(lat, ["μ2", "γ", "κ", "κ'", "γ", "μ"]; num_points=100) 
bands = getbands(H, ks, valley; format=:sparse, num_bands=25) 
nothing 

In [ ]:
# import LatticeQM.Spectrum
# import LatticeQM.TightBinding
# import LatticeQM.Operators
# import LatticeQM.Utils
# using ProgressMeter


# fermidirac_T(ϵ; T::Number=0.01) = 1.0 ./ (exp.(ϵ ./ T) .+ 1)
# fermidirac_0(ϵ) = heaviside.(-ϵ)
# fermidirac(ϵ; T=0, μ=0) = (T > 0) ? fermidirac_T(ϵ .- μ; T=T) : fermidirac_0(ϵ .- μ)

# fourierphase(k, δL) = exp(1.0im * 2π * dot(k, δL))

# function densitymatrix_serial!(ρs::TightBinding.AbstractHops, H, ks::AbstractMatrix{Float64}, kweights::AbstractVector{Float64}, μ::Float64=0.0; T::Real=0.01, progressmin::Int=20, kwargs...)

#     L = size(ks, 2)
#     energies = zeros(Float64, L)

#     δLs = keys(ρs)
#     for δL in δLs
#         ρs[δL] .= 0.0
#     end

#     fourierphases = [TightBinding.fourierphase(-k, δL) for δL in δLs, k in eachcol(ks)]

#     ρ_k = zeros(ComplexF64, size(H))

#     @showprogress progressmin "Density matrix " for i_ = 1:L
#         k = ks[:, i_]
#         w_k = kweights[i_]

#         @time "eigen" ϵs_k, U_k = Spectrum.spectrum(H(k); kwargs...)
#         fd_k = fermidirac.(real.(ϵs_k .- μ); T=T)
#         phases = fourierphases[:, i_]

#         @time "rho" mul!(ρ_k, U_k, Diagonal(fd_k) * U_k')

#         @time "loop assign" for (n_, δL) in enumerate(δLs)
#             ρs[δL] .+= w_k .* ρ_k .* phases[n_]
#         end

#         energies[i_] = real(w_k * sum(ϵs_k .* fd_k))
#     end

#     sum(energies) # return the kinetic part of the gs energy
# end


In [ ]:
# ρ1 = Meanfield.initialguess(v, :ferro; lat=lat)
# ks = Structure.regulargrid(nk=2^2)
# L = size(ks, 2); kweights = fill(1 / L, L)

# @time "densitymatrix" densitymatrix_serial!(ρ1, H, ks, kweights, μ; T=0.002, format=:dense)

In [ ]:
# ρ1 = Meanfield.initialguess(v, :ferro; lat=lat)
# ks = Structure.regulargrid(nk=6^2)
# @time "densitymatrix" Operators.getdensitymatrix!(ρ1, H, ks, μ; multimode=:serial, T=0.002, format=:dense)

In [ ]:
v = Operators.gethubbard(lat; mode=:σx, a=0.5, U=2.8)
ρ_init = Meanfield.initialguess(v, :ferro; lat=lat)

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, residue = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling; klin=6, iterations=25, tol=1e-3,# p_norm=Inf,
    T=0.002, β=0.93, show_trace=true, clear_trace=true, verbose=false, multimode=:distributed
)

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, residue = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling; klin=6, iterations=25, tol=1e-3,# p_norm=Inf,
    T=0.002, β=0.93, show_trace=true, clear_trace=true, verbose=false, multimode=:serial
)

In [ ]:
ρ_sol, ϵ_GS, HMF, converged, residue = Meanfield.solvehartreefock( # run the calculation
    H, v, ρ_init, filling; klin=6, iterations=25, tol=1e-3,# p_norm=Inf,
    T=0.002, β=0.93, show_trace=true, clear_trace=true, verbose=true, multimode=:distributed
)